In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

## Intro
- 앞서 `get_feautre.ipynb` 에서 구한 파일들을이용해 실제로 valid/ test set을 분리 해보겠습니다.

In [2]:
path = "/opt/ml/input/data"

# train_data 를 불러올 때는 dtype을 명시해줘서 불러옵니다

train_df = pd.read_csv(os.path.join(path,'train_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)


### feature_engineering
- 유저의 문제 풀이 숫자, 정답 숫저, 정답률을 시간순으로 누적해줍니다.
- 문제 풀이이 시간의 이상치(650 이상)을 제거해줍니다.

In [3]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']
    

    # diff()를 이용하여 시간 차이를 구해줍니다
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    
    # 만약 0초만에 풀었으면 0으로 치환
    diff = diff.fillna(pd.Timedelta(seconds=0))
    
    # 시간을 전부 초단위로 변경합니다.
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

    # df에 elapsed(문제 풀이 시간)을 추가해줍니다.
    df['t_elapsed'] = diff
    
    # 문제 풀이 시간이 650초 이상은 이상치로 판단하고 제거합니다.
    df['t_elapsed'] = df['t_elapsed'].apply(lambda x : x if x <650 else None)
    
    # 대분류(앞 세자리)
    df['i_head']=df['testId'].apply(lambda x : int(x[1:4])//10)

    # 중분류(중간 세자리)
    df['i_mid'] = df['testId'].apply(lambda x : int(x[-3:]))

    # 문제 번호(분류를 제외한)
    df['i_tail'] = df['assessmentItemID'].apply(lambda x : int(x[-3:]))
        
    # 만들어 놓은 피쳐 추가

    # 유저 피쳐
    userID_feature = pd.read_csv(os.path.join(path, "feature/userID_feature.csv"), index_col= 0)
    # 시험지 피쳐

    testId_feature = pd.read_csv(os.path.join(path,"feature/testId_feature.csv"), index_col= 0)
    # 태그 피쳐
    knowLedgedTag_acc = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    knowLedgedTag_acc.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, userID_feature, on=['userID', 'i_head'], how="left")
    df = pd.merge(df, testId_feature, on=['testId'], how="left")
    df = pd.merge(df, knowLedgedTag_acc, on=['KnowledgeTag'], how="left")
    
    return df

In [4]:
# 전부 붙여줍니다.
train_df = feature_engineering(train_df)
train_df.head(30)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,t_elapsed,...,u_head_mean,u_head_count,u_head_elapsed,i_mid_elapsed,i_mid_mean,i_mid_sum,i_mid_count,i_mid_tag_count,tag_mean,tag_sum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,0.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.955022,637.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.000000,3.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.913187,3040.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,2,1.000000,8.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.913187,3040.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,3.0,3,1.000000,7.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.913187,3040.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,4.0,4,1.000000,7.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.913187,3040.0
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,5.0,5,1.000000,11.0,...,0.791908,346,36.533784,20.842105,0.952667,1429.0,7,2,0.913187,3040.0
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,6.0,6,1.000000,NaN,...,0.791908,346,36.533784,47.064178,0.793714,1389.0,7,1,0.799552,3570.0
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,6.0,7,0.857143,7.0,...,0.791908,346,36.533784,47.064178,0.793714,1389.0,7,1,0.799552,3570.0
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,7.0,8,0.875000,64.0,...,0.791908,346,36.533784,47.064178,0.793714,1389.0,7,1,0.799552,3570.0
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,8.0,9,0.888889,15.0,...,0.791908,346,36.533784,47.064178,0.793714,1389.0,7,1,0.799552,3570.0


In [5]:
train_df.columns
    

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'user_correct_answer', 'user_total_answer', 'user_acc',
       't_elapsed', 'i_head', 'i_mid', 'i_tail', 'u_head_mean', 'u_head_count',
       'u_head_elapsed', 'i_mid_elapsed', 'i_mid_mean', 'i_mid_sum',
       'i_mid_count', 'i_mid_tag_count', 'tag_mean', 'tag_sum'],
      dtype='object')

### test_data도 똑같이 처리해줍니다

In [6]:
test_df = pd.read_csv(os.path.join(path,'test_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [7]:
# FEATURE ENGINEERING
test_df = feature_engineering(test_df)
test_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,t_elapsed,...,u_head_mean,u_head_count,u_head_elapsed,i_mid_elapsed,i_mid_mean,i_mid_sum,i_mid_count,i_mid_tag_count,tag_mean,tag_sum
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,NaN,0,NaN,0.0,...,0.655052,861,63.258152,68.609658,0.549714,962.0,7,4,0.587097,91.0
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,1.0,1,1.000000,26.0,...,0.655052,861,63.258152,68.609658,0.549714,962.0,7,4,0.587097,91.0
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,2.0,2,1.000000,94.0,...,0.655052,861,63.258152,68.609658,0.549714,962.0,7,4,0.588517,123.0
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,2.0,3,0.666667,5.0,...,0.655052,861,63.258152,68.609658,0.549714,962.0,7,4,0.588517,123.0
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,2.0,4,0.500000,7.0,...,0.655052,861,63.258152,68.609658,0.549714,962.0,7,4,0.581986,252.0


In [8]:
test_df.columns.tolist()

['userID',
 'assessmentItemID',
 'testId',
 'answerCode',
 'Timestamp',
 'KnowledgeTag',
 'user_correct_answer',
 'user_total_answer',
 'user_acc',
 't_elapsed',
 'i_head',
 'i_mid',
 'i_tail',
 'u_head_mean',
 'u_head_count',
 'u_head_elapsed',
 'i_mid_elapsed',
 'i_mid_mean',
 'i_mid_sum',
 'i_mid_count',
 'i_mid_tag_count',
 'tag_mean',
 'tag_sum']

In [9]:
blank = test_df[test_df['answerCode']==-1].copy()

In [10]:
len(blank['userID'])

744

### DATA SPLIT
- 이제 데이터를 나눠줍니다.

In [11]:
from collections import defaultdict


In [12]:
def get_head_mid(x):
    dic = defaultdict(set)
    for i in x:
        head = int(i[1:4])//10
        mid = int(i[-3:])
        dic[head].add(mid)
    return dic

### 비슷한 User?
- 정답률이 비슷하고
- 풀이시간이 비슷하면
- 학습정도가 유사한(비슷한 유저)라고 볼 수 있지 않을까요?

### 비슷한 학습 경험
- seq length가 유사하고(풀이 경험)
- testId가 비슷하면 비슷한 User가 아닐가요?
- ex) {3: {7}, 4: {16}} 라고 되어 있으면 3번 대분류의 7번 시험지, 4번 대분류의 16번 시험지를 푼 유저입니다.

In [13]:
# 마지막 index추출해주는 lambda 생성
# 마지막 index를 가져오면 최근까지의 누적 학습 정답률과 풀이 갯수를 가저올 수 있습니다
current = lambda x : list(x)[-1]

TRAINID = train_df.groupby(['userID']).agg({
        'testId':get_head_mid, 
        'answerCode':'count',
        't_elapsed':'mean',
        'u_head_mean':current, 
        'u_head_count':current, 
        'i_head':current, 
        'i_mid':current}).reset_index()

TRAINID.columns = ['userID', 'testId', 'u_count', 'u_elapsed', 'u_head_mean','u_current_head_count', 'u_current_head', 'u_current_mid']
TRAINID = TRAINID.sort_values(by=['u_count', 'u_current_head','u_current_mid']).reset_index(drop=True)
TRAINID[:10]

,userID,testId,u_count,u_elapsed,u_head_mean,u_current_head_count,u_current_head,u_current_mid
0,7441,"{3: {71}, 4: {165}}",9,43.750000,1.000000,4,4,165
1,7414,"{4: {194}, 3: {26}, 2: {124}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {10}, 5: {193}, 4: {132}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {74, 75}, 5: {193}}",13,22.454545,1.000000,4,5,193
4,7386,"{1: {16, 155}, 4: {132}}",14,28.000000,0.600000,10,1,155
5,6981,"{2: {194, 143}, 4: {196}}",14,26.666667,0.888889,9,2,143
6,7189,"{3: {144, 76}, 2: {179}}",14,36.833333,0.250000,4,2,179
7,7330,"{4: {95}, 1: {145}, 2: {179}}",14,22.333333,1.000000,4,2,179
8,7349,"{6: {185}, 4: {165}, 3: {126}}",14,68.500000,0.400000,5,3,126
9,7422,"{6: {93}, 3: {42, 171}}",14,20.500000,0.300000,10,3,171


In [14]:
TESTID = test_df.groupby(['userID']).agg({
        'testId':get_head_mid, 
        'answerCode':'count',
        't_elapsed':'mean',
        'u_head_mean':current, 
        'u_head_count':current, 
        'i_head':current, 
        'i_mid':current}).reset_index()

TESTID.columns = ['userID', 'testId', 'u_count', 'u_elapsed', 'u_head_mean','u_current_head_count', 'u_current_head', 'u_current_mid']
TESTID = TESTID.sort_values(by=['u_count', 'u_current_head','u_current_mid']).reset_index(drop=True)
TESTID[:10]

,userID,testId,u_count,u_elapsed,u_head_mean,u_current_head_count,u_current_head,u_current_mid
0,5965,"{3: {2, 183}, 2: {159}}",15,29.000000,0.500000,4,2,159
1,7404,"{2: {124}, 4: {109}, 3: {111}}",15,33.846154,0.500000,4,3,111
2,7033,"{4: {189, 86, 133}}",15,9.000000,0.214286,14,4,133
3,7354,"{1: {101}, 3: {144}, 4: {170}}",15,66.923077,0.250000,4,4,170
4,7373,"{3: {150}, 4: {182}, 5: {178}}",15,24.615385,1.000000,4,5,178
5,7416,"{8: {2}, 3: {160}, 5: {193}}",15,29.461538,0.666667,3,5,193
6,7417,"{1: {107, 93}, 5: {193}}",15,50.916667,0.666667,3,5,193
7,7216,"{2: {184, 170}, 3: {197}}",16,51.714286,0.400000,10,2,170
8,7439,"{4: {130, 3, 197}}",16,37.928571,0.733333,15,4,130
9,7146,"{1: {27}, 4: {192, 194}, 6: {67}}",16,38.571429,0.000000,3,6,67


### Valid 추출
TRAIN에서 TEST와 비슷한 VALID를 추출하기 위해선 다음과 같은 우선순위를 가집니다.

1. 마지막으로 풀었던 문제의 대분류가 같은가?
2. 마지막 대분류에서 풀었던 문제의 갯수가 비슷한가?(상위 30개)
3. 상위 30개중에서 중분류가 똑 같은 유저가 존재하는가?
4. 없다면 가장 정답률이 비슷한 최종 1명 추출

In [15]:
similar_dict = {}

for _, row in tqdm(TESTID.iterrows()):
    # 1. 마지막으로 풀었던 문제의 대분류가 같은가?
    similar_users = TRAINID[TRAINID['u_current_head']==row['u_current_head']].copy()

    # 선택되지 않은 것을 골라냅니다.
    similar_users = similar_users[similar_users['userID'].apply(lambda x : x not in similar_dict.values())]
    
    # 2. 마지막 대분류에서 풀었던 문제의 갯수가 비슷한가?(상위 30개)
    similar_users['u_current_head_count'] = similar_users['u_current_head_count'].apply(lambda x : abs(x-row['u_current_head_count']))
    similar_users = similar_users.sort_values(by='u_current_head_count').reset_index(drop=True)
    similar_users = similar_users[:min(similar_users.shape[0],30)].copy()
    
    # 3. 상위 30개중에서 중분류가 똑 같은 유저가 존재하는가?
    # 4. 없다면 가장 정답률이 비슷한 최종 1명 추출
    similar_users['s_mean'] = (similar_users['u_head_mean'] - row['u_head_mean']).apply(lambda x :abs(x))
    similar_users['s_mid'] = (similar_users['u_current_mid'] == row['u_current_mid']).astype(int)
    similar_users = similar_users.sort_values(by=['s_mid','s_mean']).reset_index(drop=True)
    similar_dict[row['userID']] = list(similar_users['userID'])[0]

print('done!')
print(f'please check {len(similar_dict)} == {len(set(similar_dict.values()))}')

744it [00:11, 67.00it/s] 

done!
please check 744 == 744


In [16]:
# 한번 정렬해줍니다.
val_idx = sorted(list(similar_dict.values()))

In [17]:
# Train에서 추출
valid_set = TRAINID[TRAINID["userID"].isin(val_idx)]

In [18]:
valid_set

,userID,testId,u_count,u_elapsed,u_head_mean,u_current_head_count,u_current_head,u_current_mid
1,7414,"{4: {194}, 3: {26}, 2: {124}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {10}, 5: {193}, 4: {132}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {74, 75}, 5: {193}}",13,22.454545,1.000000,4,5,193
5,6981,"{2: {194, 143}, 4: {196}}",14,26.666667,0.888889,9,2,143
8,7349,"{6: {185}, 4: {165}, 3: {126}}",14,68.500000,0.400000,5,3,126
...,...,...,...,...,...,...,...,...
6633,215,"{7: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",1326,53.961572,0.677140,923,7,138
6634,886,"{7: {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23...",1329,55.566516,0.719902,407,5,131
6636,433,"{5: {128, 129, 130, 3, 131, 5, 132, 7, 9, 137,...",1338,45.134927,0.508423,653,7,161
6667,174,"{4: {2, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18,...",1525,58.796935,0.426891,595,7,137


### Test와 valid가 유사한지 비교해보겠습니다.
- userID_test : 원래 test 유저
- userID_valid : userID_test와 비슷한 valid 유저
- current_garde_test(valid) : test(valid) 유저가 최근에 푼 대분류의 종류
- grade_acc_test(valid) : test(valid) 유저가 최근에 푼 대분류의 문제의 평균 정답률
- current_count_test(valid) : test(valid)유저가 해당 대분류에 대해 푼 문제의 개수
- count_test(valid) : test(valid)유저가 푼 전체 문제의 개수
- current_mid_test(valid) : test(valid) 유저가 최근에 푼 시험지의 종류

In [19]:
TESTID['similar'] = TESTID['userID'].apply(lambda x : similar_dict[x]) 


In [20]:
similarity = pd.merge(TESTID,valid_set,left_on='similar',right_on='userID',how='inner')


In [21]:
similarity.columns =['userID_t', 'testId_t', 'u_count_t', 'u_elapsed_t', 'u_head_mean_t', 'u_current_head_count_t', 'u_current_head_t', 'u_current_mid_t', 'similar', 
                     'userID_v', 'testId_v', 'u_count_v', 'u_elapsed_v', 'u_head_mean_v', 'u_current_head_count_v', 'u_current_head_v', 'u_current_mid_v']

### 보는 방법
- ex) 3번 test 유저와 92번 valid 유저는 비슷합니다.
    - 같은 대분류 5번을 최근에 풀었으며
    - 0.6550, 0.6530으로 정답률이 비슷합니다.
    - 최근 까지 푼 대분류 문제의 개수는 861, 588개로 비슷하지 않습니다.
    - 하지만 풀었던 모든 문제의 개수는 1036개, 973개로 비슷합니다.

In [22]:
similarity[['userID_t', 'userID_v',            
            'u_count_t', 'u_count_v',
            'u_head_mean_t', 'u_head_mean_v',
            'u_current_head_count_t', 'u_current_head_count_v',
            'u_current_head_t','u_current_head_v',
            'u_current_mid_v', 'u_current_mid_v']].sort_values(by=['userID_t']).head(30)


,userID_t,userID_v,u_count_t,u_count_v,u_head_mean_t,u_head_mean_v,u_current_head_count_t,u_current_head_count_v,u_current_head_t,u_current_head_v,u_current_mid_v,u_current_mid_v
717,3,92,1036,973,0.655052,0.653061,861,588,5,5,155,155
610,4,4045,671,886,0.703529,0.678218,425,404,7,7,134,134
738,13,868,1317,670,0.390593,0.385417,489,480,7,7,157,157
734,17,2434,1260,810,0.924757,0.921986,412,423,9,9,67,67
468,26,3636,387,349,0.817365,0.820598,334,301,6,6,134,134
675,29,926,854,1773,0.891667,0.886275,480,510,2,2,169,169
723,45,2013,1084,439,0.556075,0.556054,214,223,4,4,144,144
615,53,1615,693,703,0.511416,0.556777,657,546,4,4,142,142
662,58,1754,811,650,0.230645,0.215035,620,572,7,7,161,161
735,64,2246,1270,389,0.617486,0.627072,366,362,7,7,154,154


In [24]:
similarity[['userID_t', 'userID_v',            
            'u_count_t', 'u_count_v',
            'u_head_mean_t', 'u_head_mean_v',
            'u_current_head_count_t', 'u_current_head_count_v',
            'u_current_head_t','u_current_head_v',
            'u_current_mid_v', 'u_current_mid_v']].sort_values(by=['userID_t']).to_csv('/opt/ml/level2-dkt-level2-recsys-08/split/similar.csv')

In [63]:
sim = similarity[['userID_t', 'userID_v',            
            'u_count_t', 'u_count_v',
            'u_head_mean_t', 'u_head_mean_v',
            'u_current_head_count_t', 'u_current_head_count_v',
            'u_current_head_t','u_current_head_v',
            'u_current_mid_t', 'u_current_mid_v']]

In [64]:
a = [6102, 1522, 2672]
result = []
for n,u in enumerate(test_df.userID.unique()):
    if u in a:
        result.append(n)



In [65]:
result

[153, 281, 613]

In [66]:
5965, 7404, 7033, 7354, 7373

(5965, 7404, 7033, 7354, 7373)

In [73]:
sim.sort_values(by='userID_t')[sim.index==12]

,userID_t,userID_v,u_count_t,u_count_v,u_head_mean_t,u_head_mean_v,u_current_head_count_t,u_current_head_count_v,u_current_head_t,u_current_head_v,u_current_mid_t,u_current_mid_v
737,94,1121,1292,961,0.862847,0.867742,576,620,1,1,146,127


In [69]:
un = []

for u_t, u_v in zip(sim.userID_t, sim.userID_v):
    if sim[sim.userID_t==u_t].u_current_mid_t.values == sim[sim.userID_v==u_v].u_current_mid_v.values:
        un.append(u_t) 


### 중복된 유저는 없는가?

In [70]:
un

[]

In [97]:
similarity.userID_v.nunique() == similarity.userID_t.nunique()

True

### 모든 유저들의 대분류가 같은가?

In [98]:
count = 0
for user_t, user_v in zip(similarity.userID_t, similarity.userID_v):
    if set(similarity.u_current_head_t[similarity.userID_t==user_t]) == set(similarity.u_current_head_v[similarity.userID_v==user_v]):
        count+=1
if count == 744:print(True)
else:print(False)

True


### 대분류와 중분류가 같은 유저는?

In [100]:
count=0
for user_t, user_v in zip(similarity.userID_t, similarity.userID_v):
    if str(list(similarity.u_current_mid_t[similarity.userID_t==user_t])[0])[:-1] == str(list(similarity.u_current_mid_v[similarity.userID_v==user_v])[0])[:-1]:
        count+=1
print(f'744 개중 {count}개')


744 개중 121개


### 대분류별 평균 점수의 유사도는?

In [101]:
grade_mean = 0
for user_t, user_v in zip(similarity.userID_t, similarity.userID_v):
    t = list(similarity.u_head_mean_t[similarity.userID_t==user_t])[0]
    v = list(similarity.u_head_mean_v[similarity.userID_v==user_v])[0]
    if t == 0 or v==0:
        continue
    grade_mean += min(t,v)/max(t,v)

print(round(grade_mean/744,5)*100,'%')

95.044 %


### 새로운 파일 저장
3가지 파일을 저장할 것입니다.
- `cv_train_data.csv` : `test`, `valid`를 포함할지 포함지 않을지 결정해서 학습을 시킵니다.
- `cv_valid_data.csv` : 위 일련의 과정을 통해 비슷한 test데이터의 user들과 비슷한 유저를 추출한 파일입니다. 마지막으로 푼 문제에는 -1로 마킹 해줍니다.
- `valid_target.csv` : -1로 마킹하기전에 valid의 정답들을 저장한 파일입니다.

### cv_train_data.csv

In [102]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
train_df = pd.read_csv(os.path.join(path, 'train_data.csv') , dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

test_df = pd.read_csv(os.path.join(path, 'test_data.csv') , dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
test_df = test_df[test_df['answerCode']!= -1].copy()

In [103]:
# validation 정보를 train에 포함 시키지 않으려면 실행!!
cv_train_df = train_df[train_df["userID"].apply(lambda x : x not in val_idx)]

# test 정보를 train에 포함 시키려면 실행!
# cv_train_df = pd.concat([train_df,test_df])
cv_train_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


In [104]:
print(train_df.userID.nunique()) # 원래 train의 user 숫자
print(cv_train_df.userID.nunique()) # valid를 포함시키면 그대로 6698, 제외하면 5954 입니다.

6698
5954


### cv_valid_df.csv

In [105]:
# train에서 val_index의 유저들을 가져옵니다.
valid_df = train_df[train_df["userID"].apply(lambda x : x in val_idx)]
valid_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
26036,35,A040093001,A040000093,1,2020-01-02 23:30:13,2094
26037,35,A040093002,A040000093,0,2020-01-02 23:31:13,2094
26038,35,A040093003,A040000093,0,2020-01-02 23:31:56,2094
26039,35,A040093004,A040000093,0,2020-01-02 23:32:15,2094
26040,35,A040093005,A040000093,1,2020-01-02 23:35:06,2094


### valid_target.csv
- validation score를 구하기위해서 마지막 문제는 따로 분리 해줍니다.

In [107]:
valid_df.head(5)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
26036,35,A040093001,A040000093,1,2020-01-02 23:30:13,2094
26037,35,A040093002,A040000093,0,2020-01-02 23:31:13,2094
26038,35,A040093003,A040000093,0,2020-01-02 23:31:56,2094
26039,35,A040093004,A040000093,0,2020-01-02 23:32:15,2094
26040,35,A040093005,A040000093,1,2020-01-02 23:35:06,2094


In [108]:
valid_target = valid_df.groupby('userID').tail(1)
valid_target

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
26726,35,A010112006,A010000112,1,2020-10-26 10:15:39,6777
27262,36,A070148008,A070000148,0,2020-12-18 08:26:33,9083
32362,43,A050135007,A050000135,0,2020-10-26 07:49:41,5298
44485,59,A070160008,A070000160,1,2020-12-27 04:37:26,9122
51142,68,A080130008,A080000130,0,2020-12-13 09:18:17,8404
...,...,...,...,...,...,...
2265801,7388,A070121001,A070000121,0,2020-11-05 03:29:30,8910
2265831,7390,A050193004,A050000193,1,2020-09-29 08:35:29,10402
2265915,7397,A040169005,A040000169,1,2020-10-07 12:36:48,2118
2266183,7414,A020124005,A020000124,0,2020-09-10 03:25:08,8018


In [386]:
##################################################
'코드에 따라 inferece 에서 자동으로 처리해주므로 마킹할 필요 X'
##################################################

# valid정답 정보를 추출했으니 마지막 문제를 -1로 masking 처리합니다.
# for idx in tqdm(valid_target.index):
#     valid_df.answerCode[valid_df.index==idx] = -1

  0%|          | 0/744 [00:00<?, ?it/s]/tmp/ipykernel_124787/1733076662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.answerCode[valid_df.index==idx] = -1
100%|██████████| 744/744 [00:08<00:00, 83.48it/s]


In [139]:
# 제대로 masking되었는지 확인
# valid_df.index[valid_df.answerCode==-1].size

0

In [109]:
# column 이름 변경
valid_target = valid_target[['answerCode']]
valid_target.columns=[['target']]

In [ ]:
# 모두 포함된 데이터 생성
all_train = pd.concat([train_df, test_df])

In [110]:
# 저장
all_train.to_csv("/opt/ml/input/data/all.csv",index=False)
cv_train_df.to_csv("/opt/ml/input/data/cv_train_data.csv",index=False)
valid_df.to_csv("/opt/ml/input/data/cv_valid_data.csv",index=False)
valid_target.to_csv("/opt/ml/input/data/valid_target.csv",index=False)